<a href="https://www.kaggle.com/code/ajilalithag/cats-vs-dogs?scriptVersionId=170307494" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import pandas as pd

# Load the dataset
sampleSubmission = pd.read_csv('/kaggle/input/c/dogs-vs-cats/sampleSubmission.csv')

# Now you can work with the dataset as usual
print(sampleSubmission.head())

In [ ]:
import zipfile
import os

# Path to the zip file
test1_zip_path = '/kaggle/input/c/dogs-vs-cats/test1.zip'
train_zip_path = '/kaggle/input/c/dogs-vs-cats/train.zip'

# Directory to extract to
extract_path = '/kaggle/working/'

# Extract the test1_zip_path file
with zipfile.ZipFile(test1_zip_path, 'r') as zip_ref1:
    zip_ref1.extractall(extract_path)
    
    
# Extract the train_zip_path file
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref2:
    zip_ref2.extractall(extract_path)

    
    
# List the files in the Test and Trian directory
TestDir = os.listdir(extract_path+"test1")
TrainDir = os.listdir(extract_path+"train")

# # Print the list of files in TestDir
# for testfile in TestDir:
#     print(testfile)

# Print the list of files in TrainDir
for trainfile in TrainDir:
    print(trainfile)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Path to the image file
image_path = '/kaggle/working/test1/1634.jpg'

# Load the image
img = mpimg.imread(image_path)

# Display the image using a specific image viewer
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.show(block=False)  # Non-blocking show

# Using a custom image viewer, e.g., 'eog' for Linux, 'open' for macOS, 'start' for Windows
import os
os.system('eog {}'.format(image_path))  # Replace 'eog' with your preferred image viewer

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
# Define paths to train and test datasets
train_dir = '/kaggle/working/train/'
test_dir = '/kaggle/working/test1/'

# Define constants
img_width, img_height = 150, 150
batch_size = 32
epochs = 10

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data Augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# Load training data
train_filenames = [os.path.join(train_dir, fname) for fname in os.listdir(train_dir)]
train_labels = [1 if 'dog' in fname else 0 for fname in os.listdir(train_dir)]  # 1 for dog, 0 for cat
train_labels = [str(label) for label in train_labels]  # Convert labels to strings
train_df = pd.DataFrame({'filename': train_filenames, 'label': train_labels})

In [ ]:
# Create the generator for training data
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    x_col='filename',
                                                    y_col='label',
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs)

# Save the model
model.save('/kaggle/working/cats_vs_dogs_cnn_model.h5')

In [ ]:
# Make predictions on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_filenames = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir)]
test_df = pd.DataFrame({'filename': test_filenames})

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  x_col='filename',
                                                  y_col=None,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode=None,
                                                  shuffle=False)

predictions = model.predict(test_generator, verbose=1)

# Save predictions to a CSV file for submission to Kaggle
filenames = [os.path.basename(filename) for filename in test_generator.filenames]
predictions_df = pd.DataFrame({'id': filenames, 'label': predictions.flatten()})
predictions_df['id'] = predictions_df['id'].apply(lambda x: x.split('.')[0])
predictions_df.to_csv('/kaggle/working/cats_vs_dogs_predictions.csv', index=False)

In [ ]:
# Importing the necessary library
from tensorflow.keras.models import load_model

In [ ]:
# Load the trained model
model = load_model('/kaggle/working/cats_vs_dogs_cnn_model.h5')

# Function to preprocess the image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
# Load the trained model
model = load_model('/kaggle/working/cats_vs_dogs_cnn_model.h5')
# Test images
test_image_paths = [
    '/kaggle/working/test1/3271.jpg',
    '/kaggle/working/test1/410.jpg',
    '/kaggle/working/test1/10134.jpg',
    '/kaggle/working/test1/3804.jpg',
    '/kaggle/working/test1/10986.jpg',
    '/kaggle/working/test1/12081.jpg',
    '/kaggle/working/test1/3660.jpg',
    '/kaggle/working/test1/473.jpg',
    '/kaggle/working/test1/4326.jpg',
    '/kaggle/working/test1/10150.jpg',
    '/kaggle/working/test1/7831.jpg',
    '/kaggle/working/test1/1334.jpg',
    '/kaggle/working/test1/4797.jpg',
    '/kaggle/working/test1/9437.jpg',
    '/kaggle/working/test1/182.jpg',
    '/kaggle/working/test1/9798.jpg',
    '/kaggle/working/test1/1275.jpg',
    '/kaggle/working/test1/2197.jpg',
    '/kaggle/working/test1/10543.jpg',
    '/kaggle/working/test1/6565.jpg',
    
]

In [ ]:
# Make predictions
for image_path in test_image_paths:
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_class = "dog" if prediction[0][0] > 0.5 else "cat"
    
    # Display the image and predicted class
    img = image.load_img(image_path, target_size=(150, 150))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Predicted Class: {predicted_class}')
    plt.show()